In [4]:
import cv2
import os

video_path = 'SnapSave.io-istockphoto 1152822473 640 adpp is(360p).mp4'

output_dir = 'frames'
os.makedirs(output_dir, exist_ok=True)

video = cv2.VideoCapture(video_path)

fps = video.get(cv2.CAP_PROP_FPS)

frame_count = 0

while True:
    ret, frame = video.read()

    if not ret:
        break

    if int(video.get(cv2.CAP_PROP_POS_MSEC)) % 1000 == 0:
        frame_filename = os.path.join(output_dir, f'frame_{frame_count}.jpg')
        cv2.imwrite(frame_filename, frame)
        frame_count += 1

video.release()

print(f'Frames extracted: {frame_count}')


Frames extracted: 10


In [10]:
pip install transformers


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.5 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.5 MB 1.8 MB/s eta 0:00:05
   ----- ---------------------------------- 1.3/9.5 MB 2.0 MB/s eta 0:00:05
   -------- ------------------------------- 2.1/9.5 MB 2.7 MB/s eta 0:00:03
   ----------------- ---------------------- 4.2/9.5 MB 4.3 MB/s eta 0:00:02
   ------------------------- -------------- 6.0/9.5 MB 5.1 MB/s eta 0:00:01
   ------------------------- -------------- 6.0/9.5 MB 5.1 MB/s eta 0:00:01
   ------------------------- -------------- 6.0/9.5 MB 5.1 MB/s eta 0:00:01
   ------------------------- -------------- 6.0/9.5 MB 5.1 MB/s eta 0:00:01
   ------------------------- ---------

Anger

Disgust

Fear

Happiness

Sadness

Surprise

In [1]:
import tensorflow as tf

# List available physical devices
devices = tf.config.list_physical_devices()

print("Available devices:")
for device in devices:
    print(f"{device.device_type}: {device.name}")

# Check available GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"Number of GPUs available: {len(gpus)}")
    for i, gpu in enumerate(gpus):
        print(f"GPU {i}: {gpu.name}")
else:
    print("No GPUs available. Using CPU.")


Available devices:
CPU: /physical_device:CPU:0
No GPUs available. Using CPU.


In [11]:
from transformers import pipeline
from PIL import Image
import os
import pandas as pd

frames_dir = '/content/images/train'
emotions = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Initialize lists to store true labels and predictions
true_labels = []
predicted_labels = []

# Initialize counters for progress
total_images = 0
processed_images = 0

# Count total images
for emotion in emotions:
    emotion_dir = os.path.join(frames_dir, emotion)
    total_images += len(os.listdir(emotion_dir))

# Initialize the pipeline (ensure to have the model defined)
pipe = pipeline("image-classification", model="dima806/facial_emotions_image_detection", device=0)

def classify_image(img_path):
    img = Image.open(img_path)
    result = pipe(img)
    if result:
        predicted_emotion = result[0]['label']
    else:
        predicted_emotion = 'Unknown'
    return predicted_emotion

# Iterate through images and classify
for emotion in emotions:
    emotion_dir = os.path.join(frames_dir, emotion)
    for img_name in os.listdir(emotion_dir):
        img_path = os.path.join(emotion_dir, img_name)
        predicted_emotion = classify_image(img_path)
        true_labels.append(emotion)
        predicted_labels.append(predicted_emotion)

        # Update and print progress
        processed_images += 1
        if processed_images % 100 == 0 or processed_images == total_images:
            print(f"Processed {processed_images}/{total_images} images")

# Convert to DataFrame
df = pd.DataFrame({'true_label': true_labels, 'predicted_label': predicted_labels})

# Calculate accuracy
accuracy = sum(df['true_label'] == df['predicted_label']) / len(df)
print(f"Overall Accuracy: {accuracy * 100:.2f}%")

# Print DataFrame (optional)
print(df)


C:\Users\vikra\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
No module named 'distutils'

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Directory where your dataset is stored
frames_dir = '/content/images/train'

# Set up image data generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Split 20% of data for validation
)

# Load the training data
train_generator = train_datagen.flow_from_directory(
    directory=frames_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Load the validation data
val_generator = train_datagen.flow_from_directory(
    directory=frames_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 23060 images belonging to 7 classes.
Found 5761 images belonging to 7 classes.


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

# Load a pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 75,101,575 (286.49 MB)

 Trainable params: 51,513,863 (196.51 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator
)

# Save the model
model.save('emotion_detection_model.h5')


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


721/721 ━━━━━━━━━━━━━━━━━━━━ 387s 513ms/step - accuracy: 0.2318 - loss: 3.2451 - val_accuracy: 0.2486 - val_loss: 1.8128
Epoch 2/100
721/721 ━━━━━━━━━━━━━━━━━━━━ 360s 493ms/step - accuracy: 0.2483 - loss: 1.8204 - val_accuracy: 0.2486 - val_loss: 1.8111
Epoch 3/100
721/721 ━━━━━━━━━━━━━━━━━━━━ 350s 482ms/step - accuracy: 0.2532 - loss: 1.8220 - val_accuracy: 0.2486 - val_loss: 1.8113
Epoch 4/100
721/721 ━━━━━━━━━━━━━━━━━━━━ 394s 498ms/step - accuracy: 0.2482 - loss: 1.8092 - val_accuracy: 0.2486 - val_loss: 1.8111
Epoch 5/100
721/721 ━━━━━━━━━━━━━━━━━━━━ 362s 499ms/step - accuracy: 0.2497 - loss: 1.8121 - val_accuracy: 0.2486 - val_loss: 1.8108
Epoch 6/100
721/721 ━━━━━━━━━━━━━━━━━━━━ 345s 475ms/step - accuracy: 0.2469 - loss: 1.8174 - val_accuracy: 0.2486 - val_loss: 1.8114
Epoch 7/100
721/721 ━━━━━━━━━━━━━━━━━━━━ 363s 501ms/step - accuracy: 0.2487 - loss: 1.8184 - val_accuracy: 0.2486 - val_loss: 1.8109
Epoch 8/100
721/721 ━━━━━━━━━━━━━━━━━━━━ 347s 479ms/step - accuracy: 0.2465 - los

In [ ]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(val_generator)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')
